In [1]:
# Draws a bounding box around an object found in image
def drawRectangle(read_image, coordinates):
    # Represent all sides of a box
    image = Image.open(read_image)
    draw = ImageDraw.Draw(image)
    draw.rectangle(coordinates, outline='red')
    image.show()

In [2]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time
# </snippet_imports>

'''
Authenticate
Authenticates your credentials and creates a client.
'''
# <snippet_vars>
subscription_key = "d5d0e372874a4c11ba463a30abe853f1"
endpoint = "https://answer-sheet.cognitiveservices.azure.com/"
# </snippet_vars>
# </snippet_imports_and_vars>

# <snippet_client>
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

In [3]:
from PIL import Image, ImageDraw
read_image = open("../images/sent.jpeg", "rb")
# image = Image.open(read_image)

# draw = ImageDraw.Draw(image)
# Call API with image and raw response (allows you to get the operation location)
read_response = computervision_client.read_in_stream(read_image, raw=True)
# Get the operation location (URL with ID as last appendage)
read_operation_location = read_response.headers["Operation-Location"]
# Take the ID off and use to get results
operation_id = read_operation_location.split("/")[-1]

# Call the "GET" API and wait for the retrieval of the results
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status.lower () not in ['notstarted', 'running']:
        break
    print ('Waiting for result...')
    time.sleep(1)


Waiting for result...
Waiting for result...


In [4]:

image = Image.open(read_image)
draw = ImageDraw.Draw(image)


colorHasher={}
counter=0
# Print results, line by line
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            r=(counter%3==0)*5*(counter//3+5)+150
            g=(counter%3==1)*5*(counter//3+5)+150
            b=(counter%3==2)*5*(counter//3+5)+150
            print(line.text)
            #print(r,g,b)
            print(line.bounding_box)
            coordinates = ((line.bounding_box[0], line.bounding_box[1]),
                           (line.bounding_box[4], line.bounding_box[5]))
            draw.rectangle(coordinates, outline=(r,g,b))
            counter+=1
image.show()
print()

Reg. no : 1518102126
175 150 150
Course Code: U15ME1002R
150 175 150
Course name : Renewable Energy Sources
150 150 175
Date : 18 / 02 / 2022
180 150 150
No. of. pages : 20
150 180 150
Pant - A
150 150 180
Energy Sources Can be classified into two types
185 150 150
- Renewable , Ex: Solar, wind
150 185 150
> Non Renewable , Ex: fossil fuel , nuclear material
150 150 185
2 .
190 150 150
Environmental impacts of energy Utilization include
150 190 150
air pollution , Climate Change . water pollution , thermal
150 150 190
Pollution and Solid Waste disposal . The emission
195 150 150
of air pollutants from fossil fuel Combustion
150 195 150
is the major cause of urban air pollution.
150 150 195
Coal mining can also pollute water
200 150 150
3 .
150 200 150
Solan Constant is the total radiation from the
150 150 200
Sun per unit of time per unit of area on a theoretical
205 150 150
Surface perpendicular to the Sun'sray and at
150 205 150

